# Proposal 3

$$
\mathcal{L} = 
$$

In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
import pickle
from tqdm import tqdm
from itertools import product

In [3]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    bwt_metrics,
    forgetting_metrics,
)

from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch import cuda, device

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Libraries

In [4]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.training import MWUNV1
from base_code.seed import set_seed

# Constants

In [5]:
TORCH_DEVICE = device("cuda" if cuda.is_available() else "cpu")
N_EXPERIENCES = 10
EXPERIMENT_SEED = 1234
METRICS_SUBFOLDER = f"retraining_saturation"
METRICS_FILENAME = f"{METRICS_SUBFOLDER}/proposal_3.pkl"
set_seed(EXPERIMENT_SEED)

# Dataset and definitions

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [6]:
scenario = PermutedMNIST(N_EXPERIENCES, seed=EXPERIMENT_SEED, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [7]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

In [8]:
best_hyperparameters = {
    'lambda_e': 10.0,
    'lambda_f': 10.0,
    'eps': 0.0001
}

## Evaluation metrics definition

In [9]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    
    loggers=[InteractiveLogger()]
)

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [10]:
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100).to(TORCH_DEVICE)
optimizer = SGD(model.parameters(), lr=1e-3)
criterion = CrossEntropyLoss().to(TORCH_DEVICE)
strategy = MWUNV1(
    model,
    optimizer,
    criterion,
    train_epochs=5,
    train_mb_size=128,
    evaluator=eval_plugin,
    eval_mb_size=128,
    device=TORCH_DEVICE,
    **best_hyperparameters
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [11]:
training_exps = product(range(5), train_stream)

for (_, experience) in tqdm(training_exps):
    strategy.train(experience)
    strategy.eval(test_stream)

0it [00:00, ?it/s]

-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 116.46it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:03<00:00, 118.96it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:03<00:00, 118.71it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:03<00:00, 120.37it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:03<00:00, 119.22it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 145.22it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.7774
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 142.56it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.1318
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 142.39it/s]
>

1it [00:25, 25.30s/it]


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.1496
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = 0.0000
	StreamForgetting/eval_phase/test_stream = 0.0000
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.2158
-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 114.79it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:03<00:00, 119.48it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:03<00:00, 120.46it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 105.24it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 115.19it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 148.28it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.0197
	ExperienceForgetting/eval_phase/test_st

2it [00:51, 25.66s/it]

...


> Eval on experience 9 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp009 = -0.0128
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp009 = 0.0128
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.8956
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.0083
	StreamForgetting/eval_phase/test_stream = 0.0083
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.9045
-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 105.18it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 114.51it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:04<00:00, 111.90it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 111.63it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 106.29it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 123.84it/s]
> Eval on experience 0 (Ta

49it [21:53, 27.57s/it]


> Eval on experience 9 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp009 = -0.0123
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp009 = 0.0123
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.8961
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.0073
	StreamForgetting/eval_phase/test_stream = 0.0073
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.9061
-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 111.60it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 109.40it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:04<00:00, 111.19it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 110.13it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 110.35it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 129.58it/s]
> Eval on experience 0 (Ta

50it [22:20, 26.82s/it]


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.9162
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.0078
	StreamForgetting/eval_phase/test_stream = 0.0078
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.9065


# Get metrics

In [12]:
accuracies: dict[int, list[float]] = dict()
forgettings: dict[int, list[float]] = dict()
bwt: dict[int, list[float]] = dict()

for i in range(N_EXPERIENCES):
    filled_i = str(i).zfill(3)
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    forgettings[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    bwt[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
        

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

# Store metrics

In [13]:
pickle.dump({
    "accuracies": accuracies,
    "forgettings": forgettings,
    "bwt": bwt,
}, open(SAVED_METRICS_PATH / METRICS_FILENAME, "wb"))